In [11]:
"""

Project Title : Digital Signature
Author: Lenon Koh, Jun Hui, Kok King, Bee Leng, Wee Han, Jia Ting
Date : 11-09-2019

"""

# import GUI library
import tkinter
from tkinter import *
from tkinter import messagebox

# import cryptography library
import base64
import cryptography.exceptions
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.serialization import load_pem_public_key


class GUI:
    
    def getText(self):
        global text
        text = self.E1.get()        
        window.destroy()

    
    def __init__(self, window):
        
        # setting window frame
        frame1 = Frame(window, height = 50, bd = 0,pady = 20)
        frame1.pack(side = TOP)
        frame2 = Frame(window, height = 50, pady = 10)
        frame2.pack()
        
        # generate label and input box
        self.L1 = Label(frame1, text="Enter your filename" , font=("Arial Bold", 15), 
                   justify = LEFT).grid(row = 0, column = 0)    
        self.E1 = Entry(frame1, width=38)
        self.E1.grid(row = 1, column = 0, ipady=20)

        # Generate Button
        self.btn = Button(frame2, text="Ok",justify = CENTER, 
                     command=self.getText).grid(column=0,  row=2, padx = 5 , pady = 1, ipadx=20)
        
        self.btn2 = Button(frame2, text="Cancel", command=window.destroy,
                      justify = CENTER).grid(column=1, row=2, padx = 5, pady = 1, ipadx=15)


        
        
        

        

    
def verifying(filename):
    # Load the public key.
    with open('public.pem', 'rb') as f:
        public_key = load_pem_public_key(f.read(), default_backend())

    # Load the payload contents and the signature.
    with open(filename, 'rb') as f:
        payload_contents = f.read()
    with open('signature.sig', 'rb') as f:
        signature = base64.b64decode(f.read())

    # Perform the verification with using public key
    try:
        output=public_key.verify(
            signature,
            payload_contents,
            padding.PSS(
                mgf = padding.MGF1(hashes.SHA256()),
                salt_length = padding.PSS.MAX_LENGTH,
            ),
            hashes.SHA256(),)
        return 'Successfully Verified'
    
    except cryptography.exceptions.InvalidSignature as e:
        return 'ERROR: Verification Failed'
    

    
if __name__ == "__main__":
    
    # Create user interface
    window_size = "270x200+300+300" # set user interface window size
    title = "Digital Signature"     # set user interface header
    text = None                     # input text
    window = tkinter.Tk()
    window.geometry(window_size)   # Set GUI window Size
    window.title(title)            # Set GUI Title
    app = GUI(window) 
    window.mainloop()   
    
    input_filename = text          # Input File
    
    # Generate private and public key
    generate_key()
        
    # sender sign the document function
    print(signing(input_filename))
    
    # recipient verify the signed document
    print(verifying(input_filename))
    
    

Sign the file successfully.
Successfully Verified


In [13]:
def verifying(filename):
    # Load the public key.
    with open('public.pem', 'rb') as f:
        public_key = load_pem_public_key(f.read(), default_backend())

    # Load the payload contents and the signature.
    with open(filename, 'rb') as f:
        payload_contents = f.read()
    with open('signature.sig', 'rb') as f:
        signature = base64.b64decode(f.read())

    # Perform the verification with using public key
    try:
        output=public_key.verify(
            signature,
            payload_contents,
            padding.PSS(
                mgf = padding.MGF1(hashes.SHA256()),
                salt_length = padding.PSS.MAX_LENGTH,
            ),
            hashes.SHA256(),)
        return 'Successfully Verified'
    
    except cryptography.exceptions.InvalidSignature as e:
        return 'ERROR: Verification Failed'

    
 # recipient verify the signed document
print(verifying(input_filename))

ERROR: Verification Failed


In [ ]:
def generate_key():
    # Generate the public/private key pair.
    private_key = rsa.generate_private_key(
        public_exponent = 65537,
        key_size = 4096,
        backend = default_backend())
    
    # Save the private key to a file.
    with open('private.key', 'wb') as f:
        f.write(private_key.private_bytes(encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,encryption_algorithm=serialization.NoEncryption(),))
        
    # Save the public key to a file.
    with open('public.pem', 'wb') as f:
        f.write(private_key.public_key().public_bytes(encoding = serialization.Encoding.PEM,
                format = serialization.PublicFormat.SubjectPublicKeyInfo,))

In [ ]:
def signing(filename):
    # Load the private key. 
    with open('private.key', 'rb') as key_file: 
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password = None,
            backend = default_backend(),)

    # Load the contents of the file to be signed.
    with open(filename, 'rb') as f:
        payload = f.read()

    # Sign the payload file with private key.
    signature = base64.b64encode(
        private_key.sign(
            payload,
            padding.PSS(
                mgf = padding.MGF1(hashes.SHA256()),
                salt_length = padding.PSS.MAX_LENGTH,
            ),
            hashes.SHA256(),))   
    
    # create signed document
    with open('signature.sig', 'wb') as f:
        f.write(signature) 
    return "Sign the file successfully."